<a href="https://colab.research.google.com/github/Jdmolano2020/TesisNer/blob/main/SroieTextAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Mount Google drive to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

##Cargar Archivos Sroie

In [5]:
# limpiamos rastros del dataset y descomprimimos
!rm -rf /tmp/SROIE2019v02
!unzip /content/drive/MyDrive/archive_SROIEv2.zip -d /tmp/SROIE2019v02/

Archive:  /content/drive/MyDrive/archive_SROIEv2.zip
   creating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/pytorch_model.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/special_tokens_map.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/tokenizer_config.json  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/training_args.bin  
  inflating: /tmp/SROIE2019v02/SROIE2019/layoutlm-base-uncased/vocab.txt  
   creating: /tmp/SROIE2019v02/SROIE2019/test/
   creating: /tmp/SROIE2019v02/SROIE2019/test/box/
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469670.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X00016469671.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005200931.txt  
  inflating: /tmp/SROIE2019v02/SROIE2019/test/box/X51005230605.txt  
  inflating: /t

##Importar Librerias

In [6]:
# Instale la versión más reciente de gensim.
# De lo contrario, puede recibir el siguiente error al ejecutar naw.WordEmbsAug():
# El objeto 'Word2VecKeyedVectors' no tiene atributo 'index_to_key'
# see: https://stackoverflow.com/questions/71032760/word2veckeyedvectors-object-has-no-attribute-index-to-key
!pip install --upgrade gensim --quiet

In [7]:
# Importar gensim.
# Nota: deberá reiniciar el tiempo de ejecución para importar la versión más reciente de gensim
import gensim
print(gensim.__version__)

4.3.1


In [8]:
# Instale el tokenizador que necesita el modelo de traducción inversa
!pip install sacremoses --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [9]:
# Install the tokenizer
import sacremoses

In [10]:
# Install the nlpaug module
!pip install nlpaug --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.8 MB/s eta 0:00:00


In [11]:
# Importar el módulo nlpaug y sus métodos
import nlpaug.augmenter.word as naw
import nlpaug.flow as nafc
from nlpaug.util import Action

In [12]:
# Install Huggingface Transformers
! pip install transformers[sentencepiece] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00


In [13]:
# Install datasets
! pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00


In [14]:
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.6 MB/s eta 0:00:00


In [15]:
# Import other modules
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gc

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [16]:
import json
import random
import re
import joblib
from tqdm import tqdm

import spacy.cli
from spacy import displacy
from spacy.util import minibatch, compounding
from spacy.training import Example
from spacy.scorer import Scorer
import pathlib

from spacy.training import offsets_to_biluo_tags
from spacy.tokenizer import Tokenizer

## Crear Modelo NER

In [17]:
nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe("ner")

In [18]:
def parse(line):
    fields = line.strip().split(",")
    if len(fields) == 9:
        return fields
    else:
        return fields[:8] + [",".join(fields[8:])]

def build_text(data):
    text = " ".join(data.text)
    text = text.replace("  "," ")
    return text

def entidades_total_prueba (ruta,archivo):
  #ruta ='/tmp/SROIE2019v02/SROIE2019/train/'
  #archivo = 'X51007419197.txt'

  print(archivo)
  with open(ruta + "box/" + archivo, encoding='utf-8', errors='ignore') as f:
    ocr = f.readlines()
  data = pd.DataFrame(
    list(map(parse, ocr)),
    columns=[*(f"coor{i}" for i in range(8)), "text"]
    )
  data=data.dropna()
  #print("data",data)
  texto = build_text(data)
  #print("texto",texto)

  with open(ruta + "entities/" + archivo, encoding='utf-8') as f:
    entities = json.load(f)
  #print("entities=",entities)
  #print("type_entities=",type(entities))
  entidades = []
  for name, ent_text in entities.items():
    if len(ent_text)==0:
      continue
    else:
      ent_text=ent_text.translate(str.maketrans({"-":  r"\-",
                                                  "]":  r"\]",
                                                  "\\": r"\\",
                                                  "^":  r"\^",
                                                  "$":  r"\$",
                                                  "*":  r"\*",
                                                  ".":  r"\.",
                                                  "(":  r"\(",
                                                  ")":  r"\)" }))
    #print("ent_text=",ent_text)
    matches = re.finditer(ent_text, texto)
    local_ent = map(lambda match: (*match.span(), name), matches)
    entidades.extend(list(local_ent))
  entidades.sort()

  label_iden = {"total": False,
                "date": False,
                "address": False,
                "company": False
      }
  #print("entidades",entidades)
  entidades_n = []
  for ent in reversed(entidades):
    #print("ent=",ent)
    label= ent[2]
    if label_iden[label]==False:
      label_iden[label]=True
      entidades_n.append(ent)
    #print("entidades_n=",entidades_n)
  entidades_n.sort()
  #print("entidades_n",entidades_n)
  texto_faltantes=[]
  i=0
  pos_ini = 0
  pos_fin = 0

  for ent in entidades_n:
    if (i == 0) & (ent[0] > 0):
      pos_ini = 0
      pos_fin = ent[0]-1
      texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
      #print("if 1 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    if i < len(entidades_n)-1:
      if ent[1]+1 != entidades_n[i+1][0]:
        pos_ini = entidades_n[i][1]+1
        pos_fin = entidades_n[i+1][0]-1
        texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
        #print("if 2 ent=",ent,"pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
    i+=1

  if (pos_fin < len(texto)-1) & (entidades_n[-1][1] < len(texto)-1):
    if pos_fin > entidades_n[-1][1]:
      pos_ini = pos_fin + 1
      pos_fin = len(texto)
    else:
      pos_ini = entidades_n[-1][1]+1
      pos_fin = len(texto)
    texto_faltantes.append((pos_ini,pos_fin,texto[pos_ini:pos_fin]))
    #print("if 3 ent=",ent, "pos_ini=",pos_ini,"pos_fin=",pos_fin,"texto_f=",texto[pos_ini:pos_fin])
  entidades_faltantes = []
  #print("texto_faltantes",texto_faltantes)
  for texto_fa in texto_faltantes:
    pos_ini,pos_fin,texto_f = texto_fa
    #print("pos_ini=",pos_ini," pos_fin=",pos_fin," texto_f=",texto_f)
    #tonkens = tokenizer(texto_f)
    tonkens = texto_f.split(' ')
    pos_ini_t = pos_ini
    for token in tonkens:
      pos_fin_t = pos_ini_t + len(token)
      #print("pos_ini_t=",pos_ini_t,"pos_fin_t=",pos_fin_t,"token=",token,"textoP=",texto[pos_ini_t:pos_fin_t])
      entidades_faltantes.append((pos_ini_t,pos_fin_t,'O'))
      pos_ini_t = pos_fin_t + 1
  #print("entidades_faltantes",entidades_faltantes)
  entidades_final=entidades_n + entidades_faltantes
  entidades_final.sort()
  spacy.training.offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  tags = offsets_to_biluo_tags(nlp.make_doc(texto), entidades_final)
  #print("tags=",tags)
  #doc_p = nlp.make_doc(texto)
  #for tok in doc_p:
   #print("tok=",tok)
  entidades_final=dict(entities=entidades_final)
  entidades_final= (texto,entidades_final,entidades_n)
  #print("entidades_final=",entidades_final)
  return(entidades_final)

##Armar Dataset

In [19]:
data_dir = "/tmp/SROIE2019v02/SROIE2019/train/box"
data_dir = pathlib.Path(data_dir)
box_count = len(list(data_dir.glob('**/*.txt')))
print(box_count)
l_box=list(data_dir.glob('**/*.txt'))

data_dir = "/tmp/SROIE2019v02/SROIE2019/test/box"
data_dir = pathlib.Path(data_dir)
box_count_test = len(list(data_dir.glob('**/*.txt')))
print(box_count_test)
l_box_test=list(data_dir.glob('**/*.txt'))

626
347


In [20]:
l_box_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box))
l_box_test_archivo= list(map(lambda cadena: str(cadena)[-16:], l_box_test))

In [21]:
textos_nerd = joblib.Parallel(n_jobs=-1)(joblib.delayed(entidades_total_prueba)('/tmp/SROIE2019v02/SROIE2019/train/',archivo) for archivo in tqdm(l_box_archivo))

100%|██████████| 626/626 [02:24<00:00,  4.33it/s]


In [22]:
textos_nerd_test = joblib.Parallel(n_jobs=-1)(joblib.delayed(entidades_total_prueba)('/tmp/SROIE2019v02/SROIE2019/test/',archivo) for archivo in tqdm(l_box_test_archivo))

100%|██████████| 347/347 [00:58<00:00,  5.98it/s]


In [23]:
df = pd.DataFrame(textos_nerd_test, columns=['text', 'entidades_final', 'entidades_n'])
test_df = df[['text']]
#print(test_df)
df = pd.DataFrame(textos_nerd, columns=['text', 'entidades_final', 'entidades_n'])
train_df = df[['text']]
#print(train_df)


In [24]:
# Convert the three DataFrames to three Datasets (Aparche Arrow format)
# Convierta los tres DataFrames de datos en tres conjuntos de datos (formato de flecha de Apache) (Aparche Arrow format)

dset_test = Dataset.from_pandas( test_df)
dset_train = Dataset.from_pandas(train_df)


In [25]:
# Gather train, val, and test Datasets to have a single DatasetDict, which can be manipulated together
# Reúna conjuntos de datos de train, val, y test para tener un solo DatasetDict, que se puede manipular juntos
sroie = DatasetDict({
  'train': dset_train,
  'test': dset_test})
# Dataset.from_pandas will add an index column, which can be removed
# Dataset.from_pandas agregará una columna de índice, que se puede eliminar

In [26]:
sroie


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 626
    })
    test: Dataset({
        features: ['text'],
        num_rows: 347
    })
})

In [27]:
# Use GPU (cuda) if available
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Initiate three alternative text augmentation strategies

In [28]:
# Initiate the synonym augmentation - # Iniciar el aumento de sinónimos
aug_syn = naw.SynonymAug(
  aug_src = 'wordnet',
  aug_max = 3)

In [29]:
augmented_sroie = []
for i in train_df.index:
    lst_augment = aug_syn.augment(train_df['text'].loc[i], n = 1)
    for augment in lst_augment:
        augmented_sroie.append(augment)

df = pd.DataFrame(augmented_sroie, columns=['text'])
df_augmented = df[['text']]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [30]:
train_df_augmented = pd.concat([train_df, df_augmented], axis = 0)

In [31]:
train_df_augmented.describe()

,text
count,1252
unique,1250
top,MR. D.I.Y. (M) SDN BHD (CO.REG :860671-D) LOT ...
freq,2


In [32]:
# Initiate the contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet) augmentation
#Inicie el aumento de incrustaciones de palabras contextuales (BERT, DistilBERT, RoBERTA o XLNet)
aug_emb = naw.ContextualWordEmbsAug(
  # Other models include 'distilbert-base-uncased', 'roberta-base', etc.
  # Otros modelos incluyen 'distilbert-base-uncased', 'roberta-base', etc.
  model_path = 'distilbert-base-uncased',
  # You can also choose "insert" - También puede elegir "insertar"
  action = "substitute",
  # Use GPU
  device = 'cuda'
  )

In [33]:
# Initiate the back translation augmentation - Iniciar el aumento de traducción inversa
aug_bt = naw.BackTranslationAug(
  # Translate English to German -  Traducir inglés a alemán
  from_model_name = 'facebook/wmt19-en-de',
  # Translate German back to English - Traducir alemán de nuevo a inglés
  to_model_name = 'facebook/wmt19-de-en',
  # Use GPU
  device = 'cuda')

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# Create a function to evaluate text augmentation on model performance on test set
# Cree una función para evaluar el aumento de texto en el rendimiento del modelo en el conjunto de prueba
def evaluate_aug(aug_strategy, n, train_df, dset_test):

  # Create two lists to store augmented tweets and their correponding labels
  # Cree dos listas para almacenar tweets aumentados y sus etiquetas correspondientes
  augmented_text = []
  ##augmented_tweets_labels = []

  # Loop through the train set to create augmented tweets
  # Note: We create n augmented tweets per original tweet.
  # Recorre el juego de train para crear tweets aumentados
  # Nota: Creamos n tweets aumentados por tweet original.
  for i in train_df.index:
    if aug_strategy == 'synonym':
      lst_augment = aug_syn.augment(train_df['text'].loc[i], n = n)
    elif aug_strategy == 'embedding':
      lst_augment = aug_emb.augment(train_df['text'].loc[i], n = n)
    else:
      lst_augment = aug_bt.augment(train_df['text'].loc[i], n = n)
    for augment in lst_augment:
      augmented_text.append(augment)
      ##augmented_tweets_labels.append(train_df['label'].loc[i])

  # Zip the two lists into a list of tuples -  Comprima las dos listas en una lista de tuplas
  ##augmented_tweets_labels = list(zip(augmented_tweets, augmented_tweets_labels))

  # Convert the list of tuples to a Pandas Dataframe. -  Convierte la lista de tuplas en un marco de datos de Pandas.
  df_augmented_text = pd.DataFrame(
    augmented_text, columns = ['text'])

  # Vertically concat the train set with the augmented texts
  # Conectar verticalmente el set de train con los textos aumentados
  train_df_augmented = pd.concat([train_df, df_augmented_text], axis = 0)

  # Convert the DataFrame to a Dataset (Aparche Arrow format)
  # Convertir el marco de datos en un conjunto de datos
  dset_train_augmented = Dataset.from_pandas(train_df_augmented)
  dset_train_augmented = dset_train_augmented.remove_columns(["__index_level_0__"])
  # Gather train, val, and test Datasets to have a single DatasetDict,
  # which can be manipulated together
  # Reúna conjuntos de datos de train, val, y test para tener un solo DatasetDict,
  # que se pueden manipular juntos
  text_augmented = DatasetDict({
    'train': dset_train_augmented,
    'test': dset_test})
  #text_augmented = text_augmented.remove_columns(["__index_level_0__"])
  return(text_augmented)

In [35]:
# Evaluar el aumento de texto sinónimo
score_synonym = evaluate_aug(
  aug_strategy = 'synonym',
  n = 1,
  train_df = train_df,
  dset_test = dset_test)
print(score_synonym)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1252
    })
    test: Dataset({
        features: ['text'],
        num_rows: 347
    })
})


In [36]:
# Evaluate the embedding text augmentation
# Evaluar el aumento de texto incrustado
score_synonym = evaluate_aug(
  aug_strategy = 'embedding',
  n = 1,
  train_df = train_df,
  dset_test = dset_test)
print(score_synonym)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1252
    })
    test: Dataset({
        features: ['text'],
        num_rows: 347
    })
})


In [37]:
# Evaluate the back translation text augmentation
# Evaluar el aumento de texto de traducción inversa
score_synonym = evaluate_aug(
  aug_strategy = 'backtranslation',
  n = 1,
  train_df = train_df,
  dset_test = dset_test)
print(score_synonym)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1252
    })
    test: Dataset({
        features: ['text'],
        num_rows: 347
    })
})


In [42]:
score_synonym ['train'][0]

{'text': '32 PUB & BISTRO OWN BY CNU TRADING 78G, JALAN SS21/62, DAMANSARA UTAMA, 47400 PETALING JAYA. (GST REG. NO : 000416321536) TAX INVOICE TABLE 5 INV NO. 504233 PAX(S): 0 DATE : 10-03-2018 23:03:06 CASHIER: CHEN DESCRIPTION QTY U.PRICE TOTAL TAX HEINEKEN (5 BTL) 2 X 95.00 190.00 SR TOTAL (EXCLUDING GST): GST PAYABLE: 10.75 TOTAL (INCLUSIVE OF GST): 190.00 TOTAL: 190.00 CLOSED: 32 11-03-2018 00:32:02 SERVER: CHEN CASH : 200.00 CHANGE : 10.00 GST SUMMARY AMOUNT (RM) TAX (RM) SR (@ 6%) **** THANK YOU ****'}

In [43]:
score_synonym ['train'][626]

{'text': '32 PUB & BISTRO OWN BY CNU TRADING 78G, JALAN SS21 / 62, DAMANSARA UTAMA, 47400 PETALING JAYA. (GST REG. NO: 000416321536) TAX INVOICE TABLE 5 INV NO. 504233 PAX (S): 0 DATE: 10-03-2018 23: 03: 06 CASHIER: CHEN DESCRIPTION QTY U.PRICE TOTAL TAX HEINEKEN (5 BTL) 2 X 95.00 SERVER: CHEN CASH: 200.00 GST * (INCLUDED OF GST): 10.75 TOTAL (INCLUDED OF GST): 190.00 TOTAL: 190.00 11-03-2018 00: 02 SERVER: CHEN CASH: 200.00 GST * (INCLUDED OF GST * (INCLUDED OF GST * 10 *).'}